In [1]:
import pandas as pd
import tensorflow as tf

In [31]:
train_df = pd.read_csv('/content/drive/MyDrive/Study/Projects/Comment Toxicity/Comment Toxicity Dataset/train.csv')

In [32]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [16]:
'\n' in train_df['comment_text'][3]

True

In [22]:
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [33]:
def remove_n(comment):
  comment = comment.replace('\n', ' ')
  comment = comment.replace('\'', '')
  for p in punctuation:
    comment = comment.replace(p, ' ')
  return comment

In [34]:
train_df['comment_text'] = train_df['comment_text'].apply(remove_n)
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,Daww He matches this background colour Im see...,0,0,0,0,0,0
2,000113f07ec002fd,Hey man Im really not trying to edit war Its...,0,0,0,0,0,0
3,0001b41b1c6bb37e,More I cant make any real suggestions on imp...,0,0,0,0,0,0
4,0001d958c54c6e35,You sir are my hero Any chance you remember...,0,0,0,0,0,0


In [35]:
X = train_df['comment_text']
y = train_df.iloc[:,2:]

In [57]:
vectorizer = tf.keras.layers.TextVectorization(max_tokens=20000, output_sequence_length=1500, output_mode='int')

In [58]:
vectorizer.adapt(X)

In [59]:
vec_text = vectorizer(X)

In [62]:
vec_text.shape

TensorShape([159571, 1500])

In [63]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vec_text, y.values))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [69]:
d_len = len(dataset)
train_len = int(d_len*0.7)
val_len = int(d_len*0.2)
test_len = int(d_len*0.1)

9972
9974


In [70]:
train = dataset.take(train_len)
val = dataset.skip(train_len).take(val_len)
test = dataset.skip(train_len + val_len).take(test_len)

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [72]:
model = Sequential()
# Create the embedding layer
model.add(Embedding(20000+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer
model.add(Dense(6, activation='sigmoid'))


In [73]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          640032    
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [75]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 10461s 1s/step - loss: 0.0612 - val_loss: 0.0485


In [80]:
1-0.0612

0.9388

In [82]:
1-0.0485

0.9515

In [83]:
X_test = pd.read_csv('/content/drive/MyDrive/Study/Projects/Comment Toxicity/Comment Toxicity Dataset/test.csv')
y_test = pd.read_csv('/content/drive/MyDrive/Study/Projects/Comment Toxicity/Comment Toxicity Dataset/test_labels.csv')

In [85]:
X_test = X_test['comment_text']

In [89]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [90]:
for batch in test.as_numpy_iterator():
    # Unpack the batch
    X_true, y_true = batch
    # Make a prediction
    yhat = model.predict(X_true)

    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 156ms/step


In [91]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.818247377872467, Recall:0.6695190072059631, Accuracy:0.4714142382144928
